In [30]:
import urllib
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import string
import nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import pickle

In [22]:
#Task1
def getpage(num):
    url = "https://community.smartthings.com/latest?no_definitions=true&page="+str(num)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")
    soup_all = soup.findAll('span', class_='link-top-line')
    title = []
    url = []
    for i in soup_all:
        post = i.find(class_= "title raw-link raw-topic-link")
        title.append(post.text)
        url.append(post.get('href'))
        #print(url)
    tag = []
    tags_all = soup.findAll(class_="link-bottom-line")
    for i in tags_all:
        tag.append(i.find('span',class_='category-name').text)
    subtag = []
    subtags_all = soup.findAll('div',class_='discourse-tags')
    for i in subtags_all:
        discourse_tag = i.find('a',class_ = "discourse-tag")
        if discourse_tag:
            subtag.append(discourse_tag.text)
        else:
            subtag.append(None)
    data = pd.DataFrame({'title':title,'tag':tag,'subtag':subtag,'url':url})
    return data

In [80]:
#Task2
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
def remove_html(text):
    sup = BeautifulSoup(text,'lxml')
    html_free = sup.get_text()
    return html_free

def remove_punc(text):
    no_punc = "".join([c for c in text if c not in string.punctuation])
    return no_punc
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words
def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(i) for i in text]
    return lem_text
def word_stemmer(text):
    stem_text = [stemmer.stem(i) for i in text]
    return stem_text

def read_from_url(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    containers = soup.findAll("div", {"class":"topic-body crawler-post"})
    df = pd.DataFrame(columns=['user', 'content'])
    count = 0
    for container in containers:

        user_container = container.findAll("span", {"itemprop":"name"})
        user = user_container[0].text
        #print("User: " + user.lower())

        content_container = container.findAll("div", {"class":"post"})
        """
        This if statement should be removed once infinite scorlling bar is handled
        """
        if content_container:
            content = remove_html(content_container[0].text)
        #print("Content: ")
        #print(content)

            dfcontent = (content.lower()).replace("\t","").replace("\n"," ").replace("https ", "https")\
                .replace("…","").replace("we’re", "we are").replace("“","").replace("”","").replace("i’ll", "i will")
        #print(dfcontent)
        #print("\n")
        
        #else:
            #print("content empty, perhaps because infinite scrolling bar")

        gruber = re.compile(r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|
        gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|
        ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz
        |ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et
        |eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id
        |ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu
        |lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np
        |nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|
        Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|
        uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+
        \)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>
        ?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop
        |info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au
        |aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|
        co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|g
        f|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo
        |jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|
        mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|p
        n|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy
        |sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|
        ye|yt|yu|za|zm|zw)\b/?(?!@)))""")
        split_dfcontent = gruber.split(dfcontent)

        for i in range(0, len(split_dfcontent), 2):
            split_dfcontent[i] = remove_punc(split_dfcontent[i])

        final_dfcontent = "".join(split_dfcontent)

        df.loc[count] = [user.lower()] + [(' '.join(final_dfcontent.split())).lower()]
        count += 1

        #print((' '.join(final_dfcontent.split())).lower())
        #print("\n")
    df['stem'] = df['content']
    for i in range(len(containers)):
        #print(df['Content'][i])
        df['stem'][i] = re.split(r'\s{1,}', df['content'][i])
    df['stem'] = df['stem'].apply(lambda x : remove_stopwords(x))
    """
    
    df['stem']=df['stem'].apply(lambda x: word_lemmatizer(x))
    """
    
    df['stem'] = df['stem'].apply(lambda x: word_stemmer(x))
    return df

In [87]:
dflist = []
for i in range(100):
    df = getpage(i)
    dflist.append(df)

In [88]:
data = pd.concat(dflist).reset_index().drop(columns = ['index'])

In [89]:
data

,title,tag,subtag,url
0,Public calendar for SmartThings developer events,Groups & Events,None,https://community.smartthings.com/t/public-cal...
1,SmartThings use in a restaurant / business - R...,Projects & Stories,None,https://community.smartthings.com/t/smartthing...
2,Shutting something off after power usage excee...,SmartApps & Automations,None,https://community.smartthings.com/t/shutting-s...
3,[BETA] Community SmartApp Installer,Community Created SmartApps,smartapp_misc,https://community.smartthings.com/t/beta-commu...
4,'Fibaro Z-Wave FGK-101 Temperature & Door/Wind...,Community Created Device Types,fibaro,https://community.smartthings.com/t/fibaro-z-w...
...,...,...,...,...
2995,[RELEASE] Routines Backup - Verify/double chec...,Community Created SmartApps,smartapp_security,https://community.smartthings.com/t/release-ro...
2996,Smartthings button lag on the first press afte...,Connected Things,None,https://community.smartthings.com/t/smartthing...
2997,VID for Signal Strength?,General Discussion,developers,https://community.smartthings.com/t/vid-for-si...
2998,Wifi method - Presence Sensor,Devices & Integrations,wifi,https://community.smartthings.com/t/wifi-metho...


In [90]:
data['starter_content'] = ''
data['starter_stem'] = ''
data['reply_content'] = ''
data['reply_stem'] = ''

In [91]:
for i in range(len(data)):
    subdata = read_from_url(data['url'][i])
    starter_content = ''
    starter_stem = []
    reply_content=''
    reply_stem = []
    for k in range(len(subdata)):
        if subdata['user'][k] == subdata['user'][0]:
            starter_content += subdata['content'][k]
            starter_stem += subdata['stem'][k]
        else:
            reply_content += subdata['content'][k]
            reply_stem += subdata['stem'][k]    
    data['starter_content'][i] = starter_content
    data['starter_stem'][i] = starter_stem
    data['reply_content'][i] = reply_content
    data['reply_stem'][i] = reply_stem

In [92]:
data.to_csv('data.csv')

In [93]:
data

,title,tag,subtag,url,starter_content,starter_stem,reply_content,reply_stem
0,Public calendar for SmartThings developer events,Groups & Events,None,https://community.smartthings.com/t/public-cal...,dear community in 2015 we are going to do a be...,"[dear, commun, 2015, go, better, job, event, p...",can anyone join this thursday call or on locat...,"[anyon, join, thursday, call, locat, heavybit,..."
1,SmartThings use in a restaurant / business - R...,Projects & Stories,None,https://community.smartthings.com/t/smartthing...,this post is partially out of boredom looking ...,"[post, partial, boredom, look, someth, talk, l...",there are lots of automation systems available...,"[lot, autom, system, avail, small, busi, compa..."
2,Shutting something off after power usage excee...,SmartApps & Automations,None,https://community.smartthings.com/t/shutting-s...,hi i have a coffee grinder that my employees a...,"[hi, coffe, grinder, employe, sometim, leav, u...",look up webcore it’s the rule engine to cook u...,"[look, webcor, it’, rule, engin, cook, kind, a..."
3,[BETA] Community SmartApp Installer,Community Created SmartApps,smartapp_misc,https://community.smartthings.com/t/beta-commu...,ok i’m ready to finally share this with the co...,"[ok, i’m, readi, final, share, commun, want, s...",wow very cool can you tell us more about which...,"[wow, cool, tell, us, smartapp, support, ye, t..."
4,'Fibaro Z-Wave FGK-101 Temperature & Door/Wind...,Community Created Device Types,fibaro,https://community.smartthings.com/t/fibaro-z-w...,the fibaro fgk101 temperature doorwindow senso...,"[fibaro, fgk101, temperatur, doorwindow, senso...",geeji thanks for creating this i’m just starti...,"[geeji, thank, creat, i’m, start, play, smartt..."
...,...,...,...,...,...,...,...,...
2995,[RELEASE] Routines Backup - Verify/double chec...,Community Created SmartApps,smartapp_security,https://community.smartthings.com/t/release-ro...,introducing routines backup on request from us...,"[introduc, routin, backup, request, user, solu...",hi rboy this apps looks very promising to moni...,"[hi, rboy, app, look, promis, monitordebug, ha..."
2996,Smartthings button lag on the first press afte...,Connected Things,None,https://community.smartthings.com/t/smartthing...,i’ve had a st button very far from my hub it’s...,"[i’v, st, button, far, hub, it’, alway, 35, se...",what’s the brand and model of the plugs that y...,"[what’, brand, model, plug, got, z, wave, repe..."
2997,VID for Signal Strength?,General Discussion,developers,https://community.smartthings.com/t/vid-for-si...,anyone know of a vid in the meta data that wil...,"[anyon, know, vid, meta, data, display, signal...",cool do you have an example or link i’d like t...,"[cool, exampl, link, i’d, like, know, event, n..."
2998,Wifi method - Presence Sensor,Devices & Integrations,wifi,https://community.smartthings.com/t/wifi-metho...,hi there im wondering if there is any method t...,"[hi, im, wonder, method, identifi, iphon, lose...",i know one method is to use a custom app calle...,"[know, one, method, use, custom, app, call, ho..."
